# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# download necessary NLTK data
import nltk
# import statements
import re
import pandas as pd
from nltk.tokenize import word_tokenize

In [3]:
# load data from database
df = pd.read_sql_table('ETLPip','sqlite:///etlpipeline.db')

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.message.values
Y = df.genre.values

In [6]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [7]:
Y[0]

'direct'

In [8]:
df = pd.read_sql_table('ETLPip','sqlite:///etlpipeline.db')
X = df.message.values
y = df.genre.values

### 2. Write a tokenization function to process your text data

In [9]:
df.genre.value_counts()

news      13054
direct    10766
social     2396
Name: genre, dtype: int64

In [10]:
def tokenize(message):
    # remove punctuation and convert to lowercase
    message = re.sub(r'[^\w\s]', '', message.lower())
    # split text into words
    tokens = message.split()
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [12]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
# Define the 36 categories
categories = ['related', 'request', 'offer', 'aid_related', 'medical_help',
              'medical_products', 'search_and_rescue', 'security', 'military',
              'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
              'missing_people', 'refugees', 'death', 'other_aid',
              'infrastructure_related', 'transport', 'buildings', 'electricity',
              'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
              'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
              'other_weather', 'direct_report']

# Load the data into a Pandas DataFrame
# Replace 'df' with your DataFrame variable name
X = df['message']
y = df[categories]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [16]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
y_pred_df = pd.DataFrame(y_pred, columns=categories)
y_test_df = pd.DataFrame(y_test, columns=categories)

In [18]:
for col in categories:
    print(f"{col}:")
    print(classification_report(y_test_df[col], y_pred_df[col]))

related:
             precision    recall  f1-score   support

          0       0.62      0.36      0.46      1266
          1       0.82      0.93      0.87      3938
          2       0.67      0.60      0.63        40

avg / total       0.77      0.79      0.77      5244

request:
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      4349
          1       0.83      0.39      0.53       895

avg / total       0.88      0.88      0.86      5244

offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

aid_related:
             precision    recall  f1-score   support

          0       0.73      0.88      0.80      3113
          1       0.76      0.53      0.63      2131

avg / total       0.74      0.74      0.73      5244

medical_help:
             precision    recall  f

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.95      1.00      0.97      4974
          1       0.80      0.06      0.11       270

avg / total       0.94      0.95      0.93      5244

search_and_rescue:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5117
          1       0.71      0.09      0.17       127

avg / total       0.97      0.98      0.97      5244

security:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5156
          1       0.00      0.00      0.00        88

avg / total       0.97      0.98      0.97      5244

military:
             precision    recall  f1-score   support

          0       0.97      1.00      0.98      5089
          1       0.48      0.08      0.13       155

avg / total       0.96      0.97      0.96      5244

child_alone:
             precision    recall  f1-score   support

          0       1.00      1.00  

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
param_grid = {
    'clf__estimator__n_estimators': [10, 20, 30],
    'clf__estimator__max_depth': [None, 10, 20],
    'clf__estimator__min_samples_split': [2, 3, 4],
    'clf__estimator__criterion': ['gini', 'entropy']
}

In [20]:
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3, verbose=2)

In [21]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  39.5s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.3s remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  39.7s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total=  40.8s
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 1.3min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed: 86.3min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20, 30], 'clf__estimator__max_depth': [None, 10, 20], 'clf__estimator__min_samples_split': [2, 3, 4], 'clf__estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [22]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'clf__estimator__criterion': 'gini', 'clf__estimator__max_depth': None, 'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 30}
Best score: 0.23369254243753576


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [23]:
y_pred = grid_search.predict(X_test)

In [24]:
# Create a DataFrame from the predictions and actual values
y_pred_df = pd.DataFrame(y_pred, columns=categories)
y_test_df = pd.DataFrame(y_test, columns=categories)

# Calculate the overall accuracy of the model
accuracy = (y_pred == y_test).mean().mean()
print(f"Overall accuracy: {accuracy:.3f}")

Overall accuracy: 0.946


In [25]:
# Iterate through the columns and print classification report for each category
for col in categories:
    print(f"{col}:")
    print(classification_report(y_test_df[col], y_pred_df[col]))

related:
             precision    recall  f1-score   support

          0       0.70      0.31      0.43      1266
          1       0.81      0.96      0.88      3938
          2       0.71      0.55      0.62        40

avg / total       0.78      0.80      0.77      5244

request:
             precision    recall  f1-score   support

          0       0.89      0.99      0.94      4349
          1       0.88      0.42      0.56       895

avg / total       0.89      0.89      0.87      5244

offer:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5218
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      5244

aid_related:
             precision    recall  f1-score   support

          0       0.76      0.88      0.81      3113
          1       0.77      0.58      0.67      2131

avg / total       0.76      0.76      0.75      5244

medical_help:
             precision    recall  f

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5117
          1       1.00      0.04      0.08       127

avg / total       0.98      0.98      0.97      5244

security:
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      5156
          1       0.33      0.01      0.02        88

avg / total       0.97      0.98      0.98      5244

military:
             precision    recall  f1-score   support

          0       0.97      1.00      0.99      5089
          1       0.60      0.04      0.07       155

avg / total       0.96      0.97      0.96      5244

child_alone:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5244

avg / total       1.00      1.00      1.00      5244

water:
             precision    recall  f1-score   support

          0       0.95      1.00      0.97      4905
          1       0.87      0.26      0.40    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [26]:
import pickle

# Save the model to a file
with open("model.pkl", "wb") as file:
    pickle.dump(grid_search, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.